In [1]:
import os
os.chdir("..")
import scanpy as sc
import numpy as np
import datasets as DS
import pickle
from model import nn_model2
import evaluation as eval

d:\anaconda3\envs\scanpy\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


数据预处理

In [2]:
adata_ref = sc.read_visium(path = "data/Human_DLPFC_151673/")
adata_ref.var_names_make_unique()
adata_ref.var["mt"] = adata_ref.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_ref, qc_vars = ["mt"], inplace = True)
sc.pp.filter_genes(adata_ref, min_cells = 3)

d:\anaconda3\envs\scanpy\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
adata_q = sc.read_visium(path = "data/Human_DLPFC_151676/")
adata_q.var_names_make_unique()
adata_q.var["mt"] = adata_q.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata_q, qc_vars = ["mt"], inplace = True)
sc.pp.filter_genes(adata_q, min_cells = 3)

d:\anaconda3\envs\scanpy\lib\site-packages\anndata\_core\anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
def prefilter_specialgenes(adata):
    id_tmp1 = np.asarray([not str(name).startswith("ERCC") for name in adata.var_names], dtype = bool)
    id_tmp2 = np.asarray([not str(name).startswith("MT-") for name in adata.var_names], dtype = bool)
    id_tmp = np.logical_and(id_tmp1, id_tmp2)
    adata._inplace_subset_var(id_tmp)
prefilter_specialgenes(adata_ref)
prefilter_specialgenes(adata_q)

In [5]:
with open("human_DLPFC/spagene.pkl", "rb") as f:
    spagene = pickle.load(f)

In [6]:
import pandas as pd
ref = pd.read_csv("data/Human_DLPFC_151673/truth.txt", sep = '\t', header = None, index_col = 0)
ref.columns = ['Ground Truth']
adata_ref.obs['clusters'] = ref.loc[adata_ref.obs_names, 'Ground Truth']
adata_ref = adata_ref[~adata_ref.obs["clusters"].isna(), ].copy()

In [7]:
import pandas as pd
ref = pd.read_csv("data/Human_DLPFC_151676/truth.txt", sep = '\t', header = None, index_col = 0)
ref.columns = ['Ground Truth']
adata_q.obs['clusters'] = ref.loc[adata_q.obs_names, 'Ground Truth']
adata_q = adata_q[~adata_q.obs["clusters"].isna(), ].copy()

In [15]:
pd.Categorical(adata_ref.obs["clusters"]).codes

array([2, 0, 6, ..., 5, 6, 1], dtype=int8)

In [8]:
from torch.utils.data import TensorDataset
import torch
from datasets.utils import preprocess
adata_ref = preprocess(adata_ref, spagene)
adata_q = preprocess(adata_q, spagene)

feature = torch.as_tensor(adata_ref.to_df().values, device = "cuda:0").float()
feature = feature.view(feature.shape[0], feature.shape[1], 1)
label = pd.Categorical(adata_ref.obs["clusters"]).codes
label = torch.as_tensor(label, device = "cuda:0").long()
train_dataset = TensorDataset(feature, label)

AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:
feature = torch.as_tensor(adata_q.to_df().values, device = "cuda:0").float()
feature = feature.view(feature.shape[0], feature.shape[1], 1)
label = pd.Categorical(adata_q.obs["clusters"]).codes
label = torch.as_tensor(label, device = "cuda:0").long()
test_dataset = TensorDataset(feature, label)

训练

In [ ]:
from model.models import Customformer
model = nn_model2.NNClassifier(num_features = len(spagene)*2, hidden_sizes=[512, 64, 16], gpu_id = 0, early_stopping=True)
model.model = Customformer(64, 4, 0.02, 0.02, 64*4, len(spagene), num_classes=7)
model.fit(train_dataset)

In [ ]:
import evaluation as eval
pred, true = model.predict(test_dataset)
eval.evaluate(true, pred, verbose = True)